In [ ]:
# zlh实现，加大迭代次数，线上0.4428左右，和lxf加权融合得到最优线上结果0.4446

import gc
import os
import time
import random
import pickle
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
import catboost as cbt
import seaborn as sns

from tqdm import tqdm
from sklearn.metrics import *
from sklearn.model_selection import *
from matplotlib import pyplot as plt


seed = 2020
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
with open('./input/origin_df.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
train_df = df.iloc[:6355374]
test_df = df.iloc[-849077:]

feature_name = [i for i in df.columns if 'feature' in i]
len(feature_name)

In [ ]:
fold = 5
kf = KFold(n_splits=fold, shuffle=True, random_state=2020)

oof = np.zeros(train_df.shape[0], )
pre = np.zeros(test_df.shape[0], )
for fold, (train_index, valid_index) in enumerate(kf.split(train, train[target].astype(int).values)):
    X_train, label_train = train.iloc[train_index][feature_name], train.iloc[train_index][target].astype(int).values
    X_valid, label_valid = train.iloc[valid_index][feature_name], train.iloc[valid_index][target].astype(int).values
    
    catboost_params = {
    'iterations': 300000,
    'learning_rate': 0.01,
    'depth': 10,
    'l2_leaf_reg': 10,
    'loss_function':'RMSE',
    'eval_metric': 'RMSE',
    'task_type':'GPU',   
    'simple_ctr':'FeatureFreq',
    'combinations_ctr':'FeatureFreq',
    'random_seed': 2020,
    'early_stopping_rounds': 500,
    'use_best_model': True,
    'verbose': 1000,
}
    
    clf = cbt.CatBoostRegressor(**catboost_params)
    clf.fit(X_train, label_train, eval_set = [(X_valid, label_valid)],)
    x1 = clf.predict(X_valid)
    y1 = clf.predict(test[feature_name])
    
    oof_train[valid_index]=x1
    predictions += y1 / nfold

In [ ]:
result = test_df[['query_id','doc_id']].reset_index(drop=True)
result['predict_label'] = predictions 
result.to_csv('xgb_baseline.csv', index=False)